# Actions from Trello

This notebook tests pulling data from Trello via the trello APIs.  

It pulls back all actions over a certain period and builds a pandas dataframe with the data for a cumulative flow diagram in Plotly (so we can render in dash ultimately)  

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import os
from requests_oauthlib import OAuth1Session

Pull in all variables - these are stored as secrets and defined either in the project or passed in to codespaces when it is started. 

In [ ]:
trello_key = os.getenv('TRELLO_KEY')
trello_token = os.getenv('TRELLO_TOKEN')
trello_board_id = os.getenv('TRELLO_BOARD_ID_SHORT')

Run the query against the trello api endpoints. 

In [ ]:
uri = f"https://api.trello.com/1/boards/{trello_board_id}/actions"
filter = "since=Jun%201%202023&limit=1000&filter=createCard,updateCard"
url = uri + "?" + filter
oauthRequest = OAuth1Session(trello_key,
                    resource_owner_key=trello_token)

headers = {'Accept': "application/json"}
r = oauthRequest.get(url, headers=headers)
print(r.status_code)

Extract the results in json

In [ ]:
json_data = json.loads(r.text)

print(len(json_data))
print(json.dumps(json_data, indent = 2, sort_keys=True))

Flatten (normalise) the nested json to the bits that matter

In [ ]:
df = pd.json_normalize(json_data)

In [ ]:
df

Update the date columns to be the index and set the timezone to local. 

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df = df.tz_convert("Pacific/Auckland")

Add new blank columns for populating shortly

In [ ]:
df["Count.Landing_Space"] = np.nan
df["Count.This_Week"] = np.nan
df["Count.Next"] = np.nan
df["Count.In_Progress"] = np.nan
df["Count.Done"] = np.nan
df["Count.Archive"] = np.nan

Populate the columns with a 1 if a card moved into the column and -1 as it leaves - to help generate cumulative view 
shortly. 


In [ ]:
# If no list set (happens sometimes) set to 'This Week'
df['data.list.name'] = df['data.list.name'] .fillna('This Week')

#Increment for moving into a list
df.loc[df['data.list.name'].str.contains("Landing Space",na=False),'Count.Landing_Space'] = 1
df.loc[df['data.list.name'].str.contains("This Week",na=False),'Count.This_Week'] = 1
df.loc[df['data.list.name'].str.contains("In Progress",na=False),'Count.In_Progress'] = 1
df.loc[df['data.list.name'].str.contains("Next",na=False),'Count.Next'] = 1
df.loc[df['data.list.name'].str.contains("Done",na=False),'Count.Done'] = 1

# Comment out - may be redundant now with above steps
#Increment for moving into a list
#df.loc[df['data.listAfter.name'].str.contains("Landing Space",na=False),'Count.Landing_Space'] = 1
#df.loc[df['data.listAfter.name'].str.contains("This Week",na=False),'Count.This_Week'] = 1
#df.loc[df['data.listAfter.name'].str.contains("In Progress",na=False),'Count.In_Progress'] = 1
#df.loc[df['data.listAfter.name'].str.contains("Next",na=False),'Count.Next'] = 1
#df.loc[df['data.listAfter.name'].str.contains("Done",na=False),'Count.Done'] = 1

#Decrement for moving out of a list
df.loc[df['data.listBefore.name'].str.contains("Landing Space",na=False),'Count.Landing_Space'] = -1
df.loc[df['data.listBefore.name'].str.contains("This Week",na=False),'Count.This_Week'] = -1
df.loc[df['data.listBefore.name'].str.contains("In Progress",na=False),'Count.In_Progress'] = -1
df.loc[df['data.listBefore.name'].str.contains("Next",na=False),'Count.Next'] = -1
df.loc[df['data.listBefore.name'].str.contains("Done",na=False),'Count.Done'] = -1

In [ ]:
df

Pull back subset of data that we care about

In [ ]:
df[["id",
    "data.card.name",
    "type",
    "data.card.closed",
    "data.list.name",
    "data.listBefore.name",
    "data.listAfter.name",
    "appCreator.id",
    "appCreator.name",
    "data.card.idLabels",
    "Count.Landing_Space",
    "Count.This_Week",
    "Count.Next",
    "Count.In_Progress",
    "Count.Done",
    "Count.Archive"
    ]]

Filter to just show the last 1 most recent action per card based on a specific date. 

TODO: Count the number of cards per list


In [ ]:
df1 = df.groupby(pd.Grouper(freq='1D')).sum()
df1

TODO: Save into a dataframe


TODO: Render the data in a plotly diagram

TODO: Iterate over generate data for every day


TODO: Render the data in a plotly diagram

TODO: Iterate filling over the data from all time & rendering into a plotly diagram

Troubleshooting: Dump output of json into a pretty nested format. 

In [ ]:
print(json.dumps(json_data, indent = 2, sort_keys=True))

Export to csv for testing

In [ ]:
df.to_csv("output.csv")